In [1]:
import pandas as pd

# Classe Quiz

In [2]:
class Quiz:
    def __init__(self, nome_quiz, total_questoes):
        self.nome_quiz = nome_quiz
        self.total_questoes = total_questoes
        self.acertos = 0
        self.erros = 0
        self.duracao_total = 0
        self.user = ''
        self.tentativa = 0
        self.resultados = {}
        self.percentual_acertos = 0
        self.media_tempo_resposta = 0
        self.mensagem = ''

    
    # Função responsável por contabilizar os acertos, os erros e o tempo toda da partida avaliar_desempenho(self)
    def respostas (self, user='',  tentativa=0, resposta_correta=[], tempo_resposta=[]):

        self.tentativa = tentativa
        self.user = user
        self.duracao_total = sum(tempo_resposta)

        for resposta in resposta_correta:
            if resposta:
                self.acertos += 1
            else:
                self.erros += 1
        
        # Calculando desempenho
        self.media_tempo_resposta = self.duracao_total / self.total_questoes
        self.percentual_acertos   = (self.acertos / self.total_questoes) * 100

        # Avaliando o resultado do usuário
        if self.percentual_acertos >= 70 and self.media_tempo_resposta <= 20:
            mensagem = "APROVADO"
        else:
            mensagem = "REPROVADO"

        self.mensagem = mensagem
        

    def armazenando_dados (self):

        self.resultados[self.user,f"Tentativa {self.tentativa}"]= {
                'Quiz':                      self.nome_quiz,
                'Acertos':                   self.acertos,
                'Erros':                     self.erros,
                'Acertos(%)':                self.percentual_acertos,
                'Média tempo resposta(s)':   self.media_tempo_resposta,
                'Duração do quiz(min)':      (self.duracao_total/60),
                'Status':                    self.mensagem
            }
        
        data_resutados = pd.DataFrame(self.resultados).T

        self.acertos = 0
        self.erros = 0
        self.percentual_acertos = 0
        self.media_tempo_resposta = 0
        self.duracao_total = 0
        self.mensagem = 0

        return data_resutados
            


In [3]:
# INPUT INICIAL
nome_quiz = 'Exame - Hemograma'
qtd       = 10

quiz = Quiz(nome_quiz,qtd)

# Analise de cada tentativa

In [4]:
# Função responsável por  filtrar as informações do DataFrame
def analise_tentativa (user, data, tentativa):

        # Filtrando o DataFrame
        data_filtrado =  data.loc[(data['User_ID'] == user) & (data['Tentativa'] == f'Tentativa_{tentativa}'), 'Respostas'].iloc[0]

        respostas = []
        tempo     = []

        for questao in range(quiz.total_questoes):
                        
                respostas.append(data_filtrado[questao][0])
                tempo.append(data_filtrado[questao][1])
                
        quiz.armazenando_dados()        
        quiz.respostas(user, tentativa, respostas, tempo)
        


# Consulta

In [5]:
def consulta(user, df):
    
    tentativas = df.loc[(df['User_ID'] == user)]

    for vez in range(1, len(tentativas) + 1):
        analise_tentativa(user, df, vez)
    
    data_user  = quiz.armazenando_dados().loc[user]

    df_user  = pd.DataFrame(data_user)

    df_user.to_excel("dados_user.xlsx", index=False)
    
    #  Mostrando as informações coletadas sobre o usuário
    print(f" PROGRESSO DO USER: {user}")
    display(data_user)

# Quizzes disponiveis

In [6]:

quizzes = {
            'Q1': {'Nome': 'Exame - hemograma', 'Qtd': 10},
            'Q2': {'Nome': 'Exame - urina',     'Qtd': 10},
            'Q3': {'Nome': 'Exame - fezes',     'Qtd': 10}, 
        }

df_quizzes = pd.DataFrame(quizzes).T

# Gerando DataFrame

In [7]:
import random
import pandas as pd

# Função para gerar tentativas com condições específicas
def gerar_tentativa(num_tentativa):
    tentativa = []
    for _ in range(10):  # 10 questões no quiz
        # Na primeira tentativa, definimos 70% de chance de erro e um tempo mais longo
        if num_tentativa == 1:
            resultado = random.choices([True, False], weights=[0.3, 0.7])[0]
            tempo = random.randint(20, 40)  # Tempo aleatório entre 20 e 40 segundos
        # Nas duas últimas tentativas, garantimos mais acertos do que erros
        elif num_tentativa >= 3:
            resultado = random.choices([True, False], weights=[0.8, 0.2])[0]
            tempo = random.randint(10, 30)  # Tempo aleatório entre 10 e 30 segundos
        # Tentativas intermediárias com distribuição uniforme
        else:
            resultado = random.choice([True, False])  
            tempo = random.randint(10, 30)  # Tempo aleatório entre 10 e 30 segundos
        tentativa.append((resultado, tempo))
    return pd.Series(tentativa)

# Dicionário para armazenar as respostas dos usuários
respostas = {}

# Gerar 100 novos usuários com 4 tentativas diferentes cada
for i in range(1, 11):
    user_id = f'User_{i}'
    tentativas_usuario = {}
    for j in range(1, 5):
        nome_tentativa = f'Tentativa_{j}'
        tentativa = gerar_tentativa(j)
        tentativas_usuario[nome_tentativa] = tentativa
    respostas[user_id] = tentativas_usuario

# Criar DataFrame com os resultados das tentativas dos usuários
data = []
for user_id, tentativas_usuario in respostas.items():
    for tentativa, respostas_tentativa in tentativas_usuario.items():
        data.append([user_id, tentativa, respostas_tentativa])

# Criar DataFrame a partir dos dados coletados
df = pd.DataFrame(data, columns=['User_ID', 'Tentativa', 'Respostas'])

# Exibir as primeiras 5 linhas do DataFrame para verificar
print(df.head())


  User_ID    Tentativa                                          Respostas
0  User_1  Tentativa_1  0    (False, 38)
1    (False, 37)
2    (False,...
1  User_1  Tentativa_2  0    (False, 27)
1     (True, 13)
2    (False,...
2  User_1  Tentativa_3  0     (True, 23)
1     (True, 30)
2    (False,...
3  User_1  Tentativa_4  0     (True, 18)
1     (True, 12)
2     (True,...
4  User_2  Tentativa_1  0    (False, 31)
1    (False, 20)
2    (False,...


In [8]:
consulta('User_1',df)

 PROGRESSO DO USER: User_1


,Quiz,Acertos,Erros,Acertos(%),Média tempo resposta(s),Duração do quiz(min),Status
Tentativa 1,Exame - Hemograma,3,7,30.0,31.7,5.283333,REPROVADO
Tentativa 2,Exame - Hemograma,3,7,30.0,18.5,3.083333,REPROVADO
Tentativa 3,Exame - Hemograma,8,2,80.0,22.6,3.766667,REPROVADO
Tentativa 4,Exame - Hemograma,9,1,90.0,19.8,3.3,APROVADO


Dentro do contexto de análise de desempenho em um quiz, existem várias outras relações que podem ser exploradas além da relação entre a porcentagem de acertos e a média do tempo de resposta. Algumas possibilidades incluem:

1. **Relação entre acertos e erros**: Você pode explorar como o número de acertos se relaciona com o número de erros. Isso pode fornecer insights sobre a consistência do desempenho do usuário e áreas específicas onde eles têm mais dificuldade.

2. **Relação entre o tempo de resposta e o número de acertos**: Em vez de apenas a média do tempo de resposta, você pode investigar como o tempo de resposta para cada pergunta se relaciona com o número de acertos. Isso pode ajudar a identificar se os usuários tendem a responder mais rapidamente a perguntas que eles acertam ou erram.

3. **Análise de tendências ao longo do tempo**: Você pode analisar como o desempenho do usuário muda ao longo das diferentes tentativas do quiz. Isso pode incluir a identificação de melhorias ou piorias consistentes no desempenho, bem como a identificação de áreas específicas de melhoria ao longo do tempo.

4. **Análise de padrões de resposta**: Além de apenas o número de acertos, você pode analisar padrões específicos de respostas dos usuários, como respostas corretas em sequência, respostas corretas após erros, etc. Isso pode fornecer insights sobre a estratégia de resposta dos usuários e áreas onde eles podem estar aplicando técnicas específicas.

5. **Correlação entre métricas de desempenho e características dos usuários**: Se você tiver informações sobre os usuários, como idade, gênero, nível de educação, etc., você pode explorar a correlação entre essas características e o desempenho no quiz. Isso pode ajudar a identificar grupos de usuários que têm um desempenho particularmente bom ou ruim e entender melhor os fatores que influenciam o desempenho.

Essas são apenas algumas das possibilidades de análise que podem ser exploradas dentro do contexto de desempenho em um quiz. A escolha das análises a serem realizadas dependerá dos seus objetivos específicos e dos dados disponíveis.